<a href="https://colab.research.google.com/github/fatemafaria142/Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/Mental_Health_Advice_Generation_using_Starling_LM_7B_alpha_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [ ]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Select 3000 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(3000))

# Select the rest for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(3000, len(instruct_tune_dataset["train"])))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3000
})

In [ ]:
test_subset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 512
})

In [ ]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 3000
Test subset size: 512


In [ ]:
# Show 5 examples from the train dataset
print("Train subset examples:")
for i in range(5):
    context = train_subset['Context'][i]
    response = train_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")

Train subset examples:
Example 1:
Context: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Response: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves

In [ ]:
# Show 5 examples from the test dataset
print("\nTest subset examples:")
for i in range(5):
    context = test_subset['Context'][i]
    response = test_subset['Response'][i]
    print(f"Example {i + 1}:")
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")


Test subset examples:
Example 1:
Context: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.
Response: Hello, and thank you for your question. You may be right, your physical symptoms of getting sick to your stomach may very well be your nerves or anxiety. That is totally normal and understandable given your circumstances. Another thing that is not unusual is that you feel better when you sit and rock. That is actually a fairly common way that many people comfort themselves in times of high stress. If you are concerned that there is an actual physical illness causing these symptoms, you may want to visit your primary care provider. They may tell you that they don't see anything physically wrong with you, and at that point you may want to visit a counselor about the symptoms and your relationship experience. A vis

#### **Creating a Mental Health Support Prompt Using LangChain**

In [ ]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Create a sample prompt template
    template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: {response}<|end_of_turn|>
"""

    # Extract context and response from the sample
    example_context = sample['Context']
    example_response = sample['Response']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=template, input_variables=["context", "response"])
    single_turn_prompt = prompt_template.format(context=example_context, response=example_response)

    return single_turn_prompt

#### **Generating LangChain Prompt for a Training Subset Examples**

In [ ]:
prompt = create_prompt(train_subset[0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same 

In [ ]:
prompt = create_prompt(train_subset[1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone. Changing our feelings is like changing our thoughts - it's ha

In [ ]:
prompt = create_prompt(train_subset[2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: First thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with so

#### **Generating LangChain Prompt for a Testing Subset Examples**

In [ ]:
prompt = create_prompt(test_subset[0])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I had to put a restraining order against my ex-fiancé. He was served last night. He was mentally and verbally abusive towards me. I thought it was my nerves, but every time I stand up, I get sick. If I sit and rock, I'm fine.<|end_of_turn|>
GPT4 Correct Assistant: Hello, and thank you for your question. You may be right, your physical symptoms of getting sick to your stomach may very well be your nerves or anxiety. That is totally normal and understandable given your circumstances. Another thing that is not unusual is that you feel better when you sit and rock. That is actually a fairly common way that many people comfort themselves in times of high s

In [ ]:
prompt = create_prompt(test_subset[1])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I’m trying to make marriage work after a split. Before our split, he lied a lot and broke every promise to me. I don't think he cheated. Last month, I asked what women work with him, so he told me. Yesterday, I found out about a girl that he said he forgot about. Should I be upset?<|end_of_turn|>
GPT4 Correct Assistant: I would ask you first what made you give him another chance after he repeatedly lied and broke every promise to you?  I would imagine if he repeatedly lied to you that it will damage your ability to trust him now.  Is he in therapy? Does he recognize that he has a problem and is he trying to repair it?  Even if he truly forgot to tell 

In [ ]:
prompt = create_prompt(test_subset[2])
print(prompt)

GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: My fiancé and I broke up. He cheated on me numerous times. I kept forgiving but questioning his every move. He got tired and left.<|end_of_turn|>
GPT4 Correct Assistant: if he as cheated on you multiple times it is not healthy for you to continue seeing him.  However It takes time to heal your pain. You are not a robot that can just switch off your emotions.  Please surround yourself with people who can support and empower you. <|end_of_turn|>



### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:**  https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [ ]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently(without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [19]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
hf = HuggingFacePipeline(pipeline=pipe)

# Create a sample prompt template

template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])


chain = prompt | hf

context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

print(chain.invoke({"context": context}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(




As a mental health counselor, it's important to understand and validate the feelings and experiences of the individual seeking help. Here's how I would respond:

It's great that you've reached out for support. Recognizing the need for help is the first and most critical step in addressing mental health challenges. It's important to remember that mental health is a journey, and it's okay not to be okay. 

The feelings of worthlessness and the sleep problems you're experiencing may seem overwhelming, but with the right guidance and support, you can work through them. Here are a few steps you can take:

1. **Seek professional help**: Talk to a mental health professional like a psychologist, psychiatrist, or therapist. They can provide personalized guidance and support. They can help you understand the underlying causes of your feelings of worthlessness and provide you with coping mechanisms.

2. **Develop a healthy sleep routine**: Establishing a regular sleep schedule is crucial for yo

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [20]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [21]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [25]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [26]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

### **Training starts here**

In [27]:
trainer.train()

Step,Training Loss
10,2.201100
20,1.905600
30,1.805400
40,1.833300
50,1.772900
60,1.697400
70,1.634200
80,1.714800
90,1.795000
100,1.962400


TrainOutput(global_step=100, training_loss=1.8321986961364747, metrics={'train_runtime': 1671.7536, 'train_samples_per_second': 0.239, 'train_steps_per_second': 0.06, 'total_flos': 1.7483553570816e+16, 'train_loss': 1.8321986961364747, 'epoch': 0.35})

### **Save the model**

In [28]:
trainer.save_model("Starling_instruct_generation")

In [29]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

In [30]:
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
hf = HuggingFacePipeline(pipeline=pipe)
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

chain = prompt | hf

### **Using `Invoke`**

In [31]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
response = chain.invoke({"context": context})
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



It's really brave of you to reach out for help. It's important to recognize that mental health challenges are nothing to be ashamed of, and asking for help is a huge step in the right direction.

It's also important to recognize that mental health struggles often stem from a complex interplay of factors. These may include biological (like genetics or brain chemistry), environmental (like stressors or traumas), and personal (like individual beliefs or coping skills).

To change your feelings of worthlessness, it might help to start by identifying some of the factors that contribute to these feelings. Some common factors include:

1. Low self-esteem: This is a belief that you are unworthy or undeserving of positive feelings. It may stem from past experiences, like being criticized or judged.

2. Negative self-talk: This is the habit of constantly putting yourself down, or having a negative inner voice that criticizes your actions or thoughts. It can be very damaging to your self-esteem 

### **Using `Batch`**

In [32]:
context_1 = "It was over 20 years ago, but the pain has resurfaced again now because I have started seeing her Facebook posts about how great her life is. I feel so angry. How can I handle this?"
context_2 = "My fiancé and I broke up. He cheated on me numerous times. I kept forgiving but questioning his every move. He got tired and left."
context_3 = "I've gone to a couple therapy sessions so far and still everytime I walk in I get nervous and shaky.  Is this normal? Should I still be feeling like this?"

contexts = [context_1, context_2, context_3]

# Invoke the LangChain pipeline for multiple contexts
responses = chain.batch([{"context": ctx} for ctx in contexts])

# Print the responses
for context, response in zip(contexts, responses):
    print(f"Context: {context}")
    print(f"Response: {response}")
    print("-------------------")


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Context: It was over 20 years ago, but the pain has resurfaced again now because I have started seeing her Facebook posts about how great her life is. I feel so angry. How can I handle this?
Response: 
It's completely understandable to feel angry when seeing your estranged sister's Facebook posts. There are a few things you can try to manage your feelings and cope with the situation.

1. **Take a step back:** Consider taking a break from social media for a while. Sometimes seeing what others are doing can make us feel inadequate and trigger emotions we aren't prepared to handle. It might be helpful to take a step back and remove yourself from the triggering situation for a bit.

2. **Reflect on your feelings:** It's important to acknowledge that you're feeling angry, but it's also important to take a step back and look at why you're feeling this way. Is there a specific memory or situation from your past that is triggering this anger? Reflecting on the source of your feelings can help 

# **Conversation Buffer memory**

In [33]:
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

In [34]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [35]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nIt's great to hear that you've never contemplated suicide, and that you've had a desire to change your feelings. It's not easy to overcome feelings of worthlessness, but it is possible. Here are some tips that you might find helpful:\n\n1. **Seek Professional Help**: It's important to talk to a mental health professional. They can help you explore the causes of your feelings of worthlessness, and provide you with strategies to cope.\n\n2. **Develop Self-Compassion**: Treat yourself with kindness and understanding. Understand that it's okay to have negative thoughts, but it's not okay to dwell on them. Recognize that you are worthy of self-care and self-compassion.\n\n3. **Reframe Negative Thoughts**: Try to think about the reasons why you feel worthless. Are they accurate? Are there other ways to interpret the situation that could be more positive? It's important to challenge negative thoughts and replace them with positive affirmations.\n\n4. **Practice Gratitude**: Take time to ap

#### **Example: 02**

In [36]:
context = "I'm 15 and my girlfriend is 14. Am I a pedophile because I'm attracted to her and she's under 18?"
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm 15 and my girlfriend is 14. Am I a pedophile because I'm attracted to her and she's under 18?<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nNo, it's not correct to refer to yourself as a pedophile. Pedophilia is a mental disorder where a person is attracted to prepubescent children. You are both still very young, and you are in the early stages of exploring your sexuality. It's normal for young people to have crushes on others their age, and it's not uncommon for romantic relationships to begin in the teenage years. \n\nHowever, it's important to remember that consent is essential in any relationship. While your ages may be similar, it's still important to respect one another's boundaries and to ensure that both of you are comfortable with the physical aspects of your relationship. \n\nYou might find it helpful to talk to a school counselor, family member, or another trusted adult about your feelings. They can provide support and guidance as you navigate these early relationships and your developing sexuality."

#### **Example: 03**

In [37]:
context = "I'm always listening to my husband, but it feels like he never listens to me."
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm always listening to my husband, but it feels like he never listens to me.<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'\nIt can be frustrating when we feel like our needs aren\'t being met in a relationship. Here are some strategies you might try:\n\n1. Express your feelings to your husband using "I" statements. For example, you could say, "I feel like you don\'t listen to me." This can help your husband understand your perspective, and he may be more likely to empathize with your feelings.\n\n2. Ask your husband to share a time when he felt like you were not listening to him. This can help both of you understand each other\'s perspective better and may lead to a deeper understanding of each other\'s needs.\n\n3. Consider using a tool like active listening. Active listening involves paying full attention to the speaker, reflecting back what you heard, and asking clarifying questions. This can help you both feel heard and understood.\n\n4. Make sure you\'re both setting aside time to talk to each other without distractions. This can help you both feel more focused and present during conversations.\n\n5

In [38]:
memory.load_memory_variables({})

{'history': 'Human: I\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here.\n   I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?\nAI: \nIt\'s great to hear that you\'ve never contemplated suicide, and that you\'ve had a desire to change your feelings. It\'s not easy to overcome feelings of worthlessness, but it is possible. Here are some tips that you might find helpful:\n\n1. **Seek Professional Help**: It\'s important to talk to a mental health professional. They can help you explore the causes of your feelings of worthlessness, and provide you with strategies to cope.\n\n2. **Develop Self-Compassion**: Treat yourself with kindness and understanding. Understand that it\'s okay to have negative thoughts, but it\'s not okay to dwell on them. Recognize that you are wo

In [39]:
print(memory.buffer)

Human: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?
AI: 
It's great to hear that you've never contemplated suicide, and that you've had a desire to change your feelings. It's not easy to overcome feelings of worthlessness, but it is possible. Here are some tips that you might find helpful:

1. **Seek Professional Help**: It's important to talk to a mental health professional. They can help you explore the causes of your feelings of worthlessness, and provide you with strategies to cope.

2. **Develop Self-Compassion**: Treat yourself with kindness and understanding. Understand that it's okay to have negative thoughts, but it's not okay to dwell on them. Recognize that you are worthy of self-care and self-compa

# **Conversation Buffer Window Memory**

In [40]:
from langchain.prompts import PromptTemplate
# Create a sample prompt template
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

In [41]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferWindowMemory(k=2)
chat_llm_chain_window = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [42]:
context = "I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on."
chat_llm_chain_window.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on.<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nIt's normal to be scared of your children, especially if they've shown a pattern of blaming you for their problems. It's also normal to feel overwhelmed and stressed about the challenges your family is facing. \nFirst and foremost, it's important to take care of yourself. You can't help your children if you're not healthy and happy. Make sure you're getting enough sleep, eating well, and taking time for yourself. \nSecondly, try to communicate openly with your children. They may be acting out because they're scared and don't know how to express their feelings. Let them know that you're there for them and that you care about their well-being. \nLastly, consider seeking professional help. A mental health counselor can provide you with support and guidance as you navigate the challenges your family is facing. They can also help you find ways to communicate more effectively with your children."

#### **Example: 02**

In [43]:
context = "I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?"
chat_llm_chain_window.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nIt's understandable that your boyfriend would be upset about you spending the weekend with a former romantic partner. It's natural for people in a relationship to feel some level of possessiveness or jealousy over their partner's time and attention. However, it's important to communicate openly with your boyfriend about your feelings and intentions. It's possible that your former romantic partner is still a close friend to you, and it's okay to maintain that friendship as long as you both respect each other's boundaries. You should have a conversation with your boyfriend about why you spent the weekend with your former romantic partner, and whether or not there are any boundaries you need to set in order to maintain your friendship. It's also important to consider how your boyfriend feels about your relationship with your former romantic partner, and whether or not there are any ways you can show your boyfriend that your friendship is respectful and appropriate."

#### **Example: 03**

In [44]:
context = "I'm depressed. I have been for years. I hide it from everyone because I'm scared of the reactions I'll get. Last time I tried telling my parents, it was a huge argument about me being too young to be depressed (I’m a legal adult), calling me ungrateful, and telling me that if I can't handle things now, it's only going to get worse in the future (which is turning out to be true). It's exhausting pretending to be okay, and I don't know how much longer I can try. I'm just really tired, and sadly, I can't afford the help I need on my own."
chat_llm_chain_window.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm depressed. I have been for years. I hide it from everyone because I'm scared of the reactions I'll get. Last time I tried telling my parents, it was a huge argument about me being too young to be depressed (I’m a legal adult), calling me ungrateful, and telling me that if I can't handle things now, it's only going to get worse in the future (which is turning out to be true). It's exhausting pretending to be okay, and I don't know how much longer I can try. I'm just really tired, and sadly, I can't afford the help I need on my own.<|end_of_turn|>
GPT4 Correct Assistant: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"\nIt's really important to address your depression. It can affect all areas of your life, including your relationships, your career, and your health. If you can't afford to pay for therapy on your own, it may be helpful to look into local mental health clinics that provide a sliding scale fee. Here are some additional steps you can take to manage your depression:\n\n1) Educate yourself about depression. Understanding the symptoms, causes, and treatments can help you feel less helpless and more in control.\n\n2) Create a support network. Talking to friends and family members about your depression can help you feel less alone. It can also provide you with a sense of accountability, as you'll be more motivated to stick to your treatment plan if others are aware of it.\n\n3) Practice self-care. Make sure you're getting enough sleep, eating healthy meals, and engaging in regular exercise. All of these things can help improve your mood and energy levels.\n\n4) Try relaxation techniques. Dee

In [45]:
memory.load_memory_variables({})

{'history': "Human: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?\nAI: \nIt's understandable that your boyfriend would be upset about you spending the weekend with a former romantic partner. It's natural for people in a relationship to feel some level of possessiveness or jealousy over their partner's time and attention. However, it's important to communicate openly with your boyfriend about your feelings and intentions. It's possible that your former romantic partner is still a close friend to you, and it's okay to maintain that friendship as long as you both respect each other's boundaries. You should have a conversation with your boyfriend about why you spent the weekend with your former romantic partner, and whether or not there are any boundaries you need to set in order to maintain your friendship. It's also important to consider how your 

In [46]:
print(memory.buffer)

Human: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?
AI: 
It's understandable that your boyfriend would be upset about you spending the weekend with a former romantic partner. It's natural for people in a relationship to feel some level of possessiveness or jealousy over their partner's time and attention. However, it's important to communicate openly with your boyfriend about your feelings and intentions. It's possible that your former romantic partner is still a close friend to you, and it's okay to maintain that friendship as long as you both respect each other's boundaries. You should have a conversation with your boyfriend about why you spent the weekend with your former romantic partner, and whether or not there are any boundaries you need to set in order to maintain your friendship. It's also important to consider how your boyfriend feels